# SST modes vs. GMST regression

1. regress modes of SST variability against GMST variability

In [ ]:
import sys
sys.path.append("..")
import scipy as sp
import numpy as np
import xarray as xr
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt

In [ ]:
%matplotlib inline
%config InlineBackend.print_figure_kwargs={'bbox_inches':None}
%load_ext autoreload
%autoreload 2
%aimport - numpy - scipy - matplotlib.pyplot

In [ ]:
from paths import path_samoc
from filters import lowpass
from SST_analysis import SST_index_analysis

In [ ]:
ds1 = xr.open_dataarray(f'{path_samoc}/SST/AMO_had.nc', decode_times=False)
ds1.name = 'AMO'
df1 = ds1.to_dataframe()

In [ ]:
ds2 = xr.open_dataarray(f'{path_samoc}/SST/TPI_had.nc', decode_times=False)
ds2.name ='TPI'
df2 = ds2.to_dataframe()

In [ ]:
ds3 = xr.open_dataarray(f'{path_samoc}/SST/SOM_had.nc', decode_times=False)
ds3.name = 'SOM'
df3 = ds3.to_dataframe()

In [ ]:
ds4 = xr.open_dataset(f'{path_samoc}/GMST/GMST_yrly_had.nc', decode_times=False)
ds4['GMST'] = lowpass(ds4['GMST']-ds4['lin_fit'], 13)
df4 = ds4['GMST'].to_dataframe()

In [ ]:
df1.plot()
df2.plot()
df3.plot()
df4.plot()

In [ ]:
dfs = [df1, df2, df3, df4]
for i, df in enumerate(dfs):
    plt.plot(df.index+10000*i)

In [ ]:
df4.index = df4.index+31

In [ ]:
df = df1.join([df2, df3, df4])

In [ ]:
df.plot()

In [ ]:
df.head()

In [ ]:
np.all(df.GMST.index==df.AMO.index)

In [ ]:
plt.plot(df.AMO.index)
plt.plot(df.SOM.index)
plt.plot(df.AMO.values)
plt.plot(df.SOM.values)

In [ ]:
plt.scatter(df.GMST, df.AMO)
plt.scatter(df.GMST, df.TPI)
plt.scatter(df.GMST, df.SOM)

In [ ]:
from itertools import combinations 

In [ ]:
pd.plotting.scatter_matrix(df, figsize=(10, 10), diagonal='kde');

In [ ]:
for n in np.arange(1,4):
    for index in combinations(['AMO', 'TPI', 'SOM'], n):
        X = sm.add_constant(df.dropna(axis=0)[list(index)])
        y = df.dropna(axis=0).GMST.dropna()
        model = sm.OLS(y, X).fit()
#         print(index, '\n', model.summary())
        print(f'{index}    R^2: {model.rsquared:4.2e} \n params:\n{model.params}\n')